In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

input_ = pd.read_excel('./1.manufacturer.xlsx',
                       header=0,
                       dtype=str).fillna('')

length = len(input_)

print('总数量：' + str(length))
print()

# = = = = = = = = = = = = = = =

import requests
import json

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for i in range(length):
    work.put_nowait(i)

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        i = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for _ in range(101):
            try:
                payload = {'func': 'partsearchgetpartgroupsdd',
                           'mfr': input_.loc[i, 'Manufacturer'],
                           'api_json_request': '1'}

                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=payload,
                                     headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text

                resp = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                html = resp['html_replace_sections']['ddrepl[partgroup_][container]']
                
                soup = BeautifulSoup(html, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_category = [category.strip() for category in html.xpath('//select[@id="partgroup_"]/option/text()')[1:]]

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
                
            except:
                time.sleep(0.3)
                continue

        if crawler_status == 'ok':
            df_temp = pd.DataFrame({'Tab_Label': input_.loc[i, 'Tab_Label'],
                                    'No.': int(input_.loc[i, 'No.']),
                                    'Manufacturer': input_.loc[i, 'Manufacturer'],
                                    'Category': list_category})

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
        else:
            df_temp = pd.DataFrame([{'Tab_Label': input_.loc[i, 'Tab_Label'],
                                     'No.': int(input_.loc[i, 'No.']),
                                     'Manufacturer': input_.loc[i, 'Manufacturer']}])
    
            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        print(input_.loc[i, 'Manufacturer'] +  '  <->  [' + crawler_status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['No.', 'Category'], ascending=[True, True])
output_correct.to_excel('./2.category-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True])
    output_error.to_excel('./category_error.xlsx', index=False)
    print()
    print('爬虫存在error')
print()
print('爬虫结束')

总数量：547

AIRAID  <->  [ok] - 剩余数量：527 - 当前时间：17:19:17
ACI  <->  [ok] - 剩余数量：526 - 当前时间：17:19:17
AEM INDUCTION  <->  [ok] - 剩余数量：525 - 当前时间：17:19:17
AGILITY  <->  [ok] - 剩余数量：524 - 当前时间：17:19:17
ADVAN-TECH  <->  [ok] - 剩余数量：523 - 当前时间：17:19:18
ACCEL  <->  [ok] - 剩余数量：522 - 当前时间：17:19:18
ACME AUTO  <->  [ok] - 剩余数量：521 - 当前时间：17:19:18
ACDELCO  <->  [ok] - 剩余数量：520 - 当前时间：17:19:18
3D  <->  [ok] - 剩余数量：519 - 当前时间：17:19:18
AIR PRO  <->  [ok] - 剩余数量：518 - 当前时间：17:19:18
ADVICS  <->  [ok] - 剩余数量：517 - 当前时间：17:19:18
AIRQUALITEE  <->  [ok] - 剩余数量：516 - 当前时间：17:19:18
ACC  <->  [ok] - 剩余数量：515 - 当前时间：17:19:18
AGS  <->  [ok] - 剩余数量：514 - 当前时间：17:19:18
AAE  <->  [ok] - 剩余数量：513 - 当前时间：17:19:18
AL'S LINER  <->  [ok] - 剩余数量：512 - 当前时间：17:19:18
AKEBONO  <->  [ok] - 剩余数量：511 - 当前时间：17:19:18
ALBANY  <->  [ok] - 剩余数量：510 - 当前时间：17:19:18
AL-KO  <->  [ok] - 剩余数量：509 - 当前时间：17:19:18
ALTROM  <->  [ok] - 剩余数量：508 - 当前时间：17:19:18
AMS AUTOMOTIVE  <->  [ok] - 剩余数量：507 - 当前时间：17:19:19
ANCHOR  <->  [ok] - 剩余数量：506 

APWI  <->  [ok] - 剩余数量：355 - 当前时间：17:19:49
FLENNOR  <->  [ok] - 剩余数量：354 - 当前时间：17:19:50
FILKO  <->  [ok] - 剩余数量：353 - 当前时间：17:19:50
EASTERN INDUSTRIES  <->  [ok] - 剩余数量：352 - 当前时间：17:19:50
FORD  <->  [ok] - 剩余数量：351 - 当前时间：17:19:50
FERODO  <->  [ok] - 剩余数量：350 - 当前时间：17:19:51
FRIGETTE  <->  [ok] - 剩余数量：349 - 当前时间：17:19:51
FAB FOURS  <->  [ok] - 剩余数量：348 - 当前时间：17:19:51
FILTRAN  <->  [ok] - 剩余数量：347 - 当前时间：17:19:51
FACET  <->  [ok] - 剩余数量：346 - 当前时间：17:19:51
COMPRESSORWORKS, INC.  <->  [ok] - 剩余数量：345 - 当前时间：17:19:52
FDP BRAKES  <->  [ok] - 剩余数量：344 - 当前时间：17:19:52
FOREIGN PARTS DISTRIBUTORS  <->  [ok] - 剩余数量：343 - 当前时间：17:19:52
FULLRIVER  <->  [ok] - 剩余数量：342 - 当前时间：17:19:53
GARNER  <->  [ok] - 剩余数量：341 - 当前时间：17:19:53
FEDERATED  <->  [ok] - 剩余数量：340 - 当前时间：17:19:53
FRICTION MASTER  <->  [ok] - 剩余数量：339 - 当前时间：17:19:53
GATES  <->  [ok] - 剩余数量：338 - 当前时间：17:19:53
FOUR SEASONS  <->  [ok] - 剩余数量：337 - 当前时间：17:19:54
FRAM  <->  [ok] - 剩余数量：336 - 当前时间：17:19:54
FRENKIT  <->  [ok] - 剩余数量：335 

OHIO CALIPER  <->  [ok] - 剩余数量：184 - 当前时间：17:20:32
PERFORMAX  <->  [ok] - 剩余数量：183 - 当前时间：17:20:32
PERMATEX  <->  [ok] - 剩余数量：182 - 当前时间：17:20:33
PACESETTER  <->  [ok] - 剩余数量：181 - 当前时间：17:20:33
PETERSON LIGHTING  <->  [ok] - 剩余数量：180 - 当前时间：17:20:33
MSD IGNITION  <->  [ok] - 剩余数量：179 - 当前时间：17:20:33
PEAK  <->  [ok] - 剩余数量：178 - 当前时间：17:20:34
MOBIL  <->  [ok] - 剩余数量：177 - 当前时间：17:20:34
PERFORMANCE TOOL  <->  [ok] - 剩余数量：176 - 当前时间：17:20:34
PERFORMANCE FRICTION  <->  [ok] - 剩余数量：175 - 当前时间：17:20:34
PIERBURG  <->  [ok] - 剩余数量：174 - 当前时间：17:20:35
PILOT AUTOMOTIVE  <->  [ok] - 剩余数量：173 - 当前时间：17:20:35
PENTOSIN  <->  [ok] - 剩余数量：172 - 当前时间：17:20:35
PIONEER  <->  [ok] - 剩余数量：171 - 当前时间：17:20:35
PENNZOIL  <->  [ok] - 剩余数量：170 - 当前时间：17:20:35
NEEDA  <->  [ok] - 剩余数量：169 - 当前时间：17:20:35
POWER TRAIN COMPONENTS  <->  [ok] - 剩余数量：168 - 当前时间：17:20:36
PIAA  <->  [ok] - 剩余数量：167 - 当前时间：17:20:36
POWER BRAKE EXCHANGE  <->  [ok] - 剩余数量：166 - 当前时间：17:20:36
PIERCE  <->  [ok] - 剩余数量：165 - 当前时间：17:20:37
PLA

VICTOR REINZ  <->  [ok] - 剩余数量：16 - 当前时间：17:21:10
VEMO  <->  [ok] - 剩余数量：15 - 当前时间：17:21:10
US MOTOR WORKS  <->  [ok] - 剩余数量：14 - 当前时间：17:21:11
ULTRA-POWER  <->  [ok] - 剩余数量：13 - 当前时间：17:21:11
VOLKSWAGEN  <->  [ok] - 剩余数量：12 - 当前时间：17:21:11
UNITED MOTOR PRODUCTS  <->  [ok] - 剩余数量：11 - 当前时间：17:21:11
VALEO  <->  [ok] - 剩余数量：10 - 当前时间：17:21:12
WAI GLOBAL  <->  [ok] - 剩余数量：9 - 当前时间：17:21:12
WALKER PRODUCTS  <->  [ok] - 剩余数量：8 - 当前时间：17:21:12
VC  <->  [ok] - 剩余数量：7 - 当前时间：17:21:13
WESTIN  <->  [ok] - 剩余数量：6 - 当前时间：17:21:13
TRAKMOTIVE  <->  [ok] - 剩余数量：5 - 当前时间：17:21:13
SUPERWINCH  <->  [ok] - 剩余数量：4 - 当前时间：17:21:14
WAGNER  <->  [ok] - 剩余数量：3 - 当前时间：17:21:14
WOLVERINE  <->  [ok] - 剩余数量：2 - 当前时间：17:21:14
WJB / INMOTION PARTS  <->  [ok] - 剩余数量：1 - 当前时间：17:21:15
WORLD BRAKE  <->  [ok] - 剩余数量：0 - 当前时间：17:21:15
WESTLING  <->  [ok] - 剩余数量：0 - 当前时间：17:21:16
TEXACO/HAVOLINE  <->  [ok] - 剩余数量：0 - 当前时间：17:21:16
XRF CHASSIS  <->  [ok] - 剩余数量：0 - 当前时间：17:21:16
WIX  <->  [ok] - 剩余数量：0 - 当前时间：17:21:17
ZF 